# Síntesis de algunas funciones RLC
<img src="./logo_UTN.svg" align="right" width="150" /> 

#### Por Mariano Llamedo Soria

### Resumen 
bla.
* Funciones de dibujo de redes: [dibujar_foster_serie](), [dibujar_foster_derivacion](), [dibujar_cauer_RC_RL]()
* Funciones de síntesis de dipolos: [cauer_RC](), [foster](), [foster_zRC2yRC]()
* Funciones para presentación de markdown y latex: [print_latex](), [print_subtitle](), [a_equal_b_latex_s]()


### Introducción

Concluye la sección de síntesis de funciones de excitación dejando algunas pautas para resolver la síntesis de redes R-L-C. Como se explicó en los documentos anteriores para las redes compuestas por dos elementos diferentes, habrá que prestar especial atención en todos los pasos de la síntesis de que la inmitancia remanente no pierda su **condición de ser realizable**, es decir que sea FRP. 

Se presentan a continuación algunos ejemplos orientados a resolver ciertos problemas especialmente útiles al momento de sintetizar **transferencias cargadas**. Se remite al lector interesado en métodos sistemáticos a la [bibliografía](https://en.wikipedia.org/wiki/Network_synthesis) (Brune, Bott-Duffin, Miyata, etc.)

Se inicializa el entorno de trabajo para comenzar con el primer ejemplo

In [3]:

import sympy as sp

# Ahora importamos las funciones de PyTC2

from pytc2.sintesis_dipolo import cauer_RC, foster, foster_zRC2yRC
from pytc2.dibujar import dibujar_cauer_RC_RL, dibujar_foster_derivacion, dibujar_foster_serie
from pytc2.general import print_latex, print_subtitle, a_equal_b_latex_s
from IPython.display import display,  Markdown


#### Ejemplo 1:

Sea la siguiente función de impedancia:


In [10]:
# Resolución simbólica
s = sp.symbols('s ', complex=True)

# Sea la siguiente función de excitación
ZZ = (s**2 + 2*s + 2)/(s**2 + 1*s + 1)

print_subtitle('Impedancia $Z_{RLC}$ ')

print_latex(a_equal_b_latex_s('Z_{RLC}(s)', ZZ))


#### Impedancia $Z_{RLC}$ 

<IPython.core.display.Math object>

In [11]:
# Se expande ZRC a la Foster
k0, koo, ki_wi, kk, ZRLC_foster = foster(ZZ)

print_subtitle('Expansión Foster de $Z_{RLC}$ ')

print_latex(a_equal_b_latex_s(a_equal_b_latex_s('Z_{RLC}(s)', ZZ)[1:-1], ZRLC_foster ))

print_latex(a_equal_b_latex_s('k_0', k0))

print_latex(a_equal_b_latex_s(r'k_1', ki_wi ))

print_latex(a_equal_b_latex_s('k_k', kk))



#### Expansión Foster de $Z_{RLC}$ 

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [12]:
# Se expande ZRC a la Foster
k0, koo, ki_wi, kk, YRLC_foster = foster(1/ZZ)

print_subtitle('Expansión Foster de $Y_{RLC}$ ')

print_latex(a_equal_b_latex_s(a_equal_b_latex_s('Y_{RLC}(s)', 1/ZZ)[1:-1], YRLC_foster ))

print_latex(a_equal_b_latex_s('k_0', k0))

print_latex(a_equal_b_latex_s(r'k_1', ki_wi ))

print_latex(a_equal_b_latex_s('k_k', kk))


#### Expansión Foster de $Y_{RLC}$ 

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

como se puede apreciar, ninguna de las formas Foster pareciera llegar a buen puerto para la síntesis, probaremos con Cauer finalmente

In [ ]:
from pytc2.sintesis_dipolo import cauer_RC
from pytc2.dibujar import dibujar_cauer_RC_RL

# Implementaremos FF mediante Cauer 1 o remociones continuas en infinito
koo, ZZ_cauer_oo, rem = cauer_RC(ZZ, remover_en_inf=True)

print_subtitle('Implementación escalera de $Z_{RLC}$ e $Y_{RLC}$')

print_latex(a_equal_b_latex_s(a_equal_b_latex_s('$ Z_{RLC}(s)', ZZ)[1:-1], ZZ_cauer_oo ))

# Tratamos a nuestra función inmitancia como una Z
dibujar_cauer_RC_RL(koo, z_exc = ZZ_cauer_oo)

print_latex(a_equal_b_latex_s(a_equal_b_latex_s('$ Y_{RLC}(s)', ZZ)[1:-1], ZZ_cauer_oo ))

# Tratamos a nuestra función inmitancia como una Y
dibujar_cauer_RC_RL(koo, y_exc = ZZ_cauer_oo)



In [ ]:
print_subtitle('Implementación circuital $Y_{RC}$ Foster serie')

# Recalculamos
k0, koo, ki_wi, kk, YRC_foster = foster(YRC/s)
k0, koo, ki_wi, kk, YRC_foster = foster_zRC2yRC(k0, koo, ki_wi, kk, YRC_foster)

# Tratamos a nuestra función imitancia como una Y
dibujar_foster_derivacion(k0, koo, ki_wi, kk, y_exc = YRC_foster)


### Implementación de  FE con inductores

Ahora presentamos las versiones equivalentes de las FE disipativas analizadas hasta ahora

In [ ]:
print_subtitle('Implementación circuital $Y_{RC}$ como $Z_{RL}$')

# Tratamos a nuestra función imitancia como una Y
dibujar_foster_serie(k0, koo, ki_wi, kk, z_exc = YRC_foster)


In [ ]:
print_subtitle('Implementación circuital $Z_{RC}$ como $Y_{RL}$')

k0, koo, ki_wi, kk, ZRC_foster = foster(ZRC)

# Tratamos a nuestra función imitancia como una Y
dibujar_foster_derivacion(k0, koo, ki_wi, kk, y_exc = ZRC_foster)


### Expansión en fracciones continuas: Método de Cauer

Del mismo modo que el método de Foster se sutenta en la expansión en fracciones simples, el de Cauer consiste en la expansión en [fracciones continuas](https://en.wikipedia.org/wiki/Network_synthesis#Cauer_synthesis). En el caso de FE disipativas, se realizará la expansión mediante los residuos $k_0$, $k_\infty$ y $k_K$, dependiendo si se expande una $Z_{RC}$ o una $Y_{RC}$:

$$ Z_{RC}(s)= \frac{1}{s.C_1} + \frac{1}{ \frac{1}{R_1} + \frac{1}{ \frac{1}{s.C_2} + \cdots } } = 
         R_1 + \frac{1}{ s.C_1 + \frac{1}{ R_2 + \cdots } } $$

$$ Y_{RC}(s)= s.C_1 + \frac{1}{ R_1 + \frac{1}{ s.C_2 + \cdots } } = 
         \frac{1}{R_1} + \frac{1}{ s.C_1 + \frac{1}{ \frac{1}{R_2} + \cdots } } $$

se ve claramente como sendas funciones de inmitancia, se corresponden con 2 redes diferentes, dependiendo dónde se realicen las remociones. Se darán ejemplos en los siguientes paneles.



## Cauer 1: remociones en $\infty$

Se comienza con la primera forma del método, es decir cuando se realizan remociones de los residuos en infinito.

In [ ]:
from pytc2.sintesis_dipolo import cauer_RC
from pytc2.dibujar import dibujar_cauer_RC_RL

# Implementaremos FF mediante Cauer 1 o remociones continuas en infinito
koo, ZRC_cauer_oo, rem = cauer_RC(ZRC, remover_en_inf=True)

print_subtitle('Implementación escalera de $Z_{RC}$ e $Y_{RL}$')

print_latex(a_equal_b_latex_s(a_equal_b_latex_s('$ Z_{RC}(s)', ZRC)[1:-1], ZRC_cauer_oo ))

# Tratamos a nuestra función inmitancia como una Z
dibujar_cauer_RC_RL(koo, z_exc = ZRC_cauer_oo)

print_latex(a_equal_b_latex_s(a_equal_b_latex_s('$ Y_{RL}(s)', ZRC)[1:-1], ZRC_cauer_oo ))

# Tratamos a nuestra función inmitancia como una Y
dibujar_cauer_RC_RL(koo, y_exc = ZRC_cauer_oo)



Se observa, como es de esperarse, que interpretar la inmitancia como impedancia o admitancia utilizando Cauer 1, da lugar a redes **duales**. Sin embargo en ambos casos se observan transferencias pasabajo (respecto a un posible puerto de salida en el extremo derecho de la red), ya que se obtienen inductores en serie y capacitores en derivación.

## Cauer 2: remociones en 0 o DC

Repetimos ahora para remociones de los residuos en 0 Hz, obtendremos dos redes también duales.

In [ ]:

# Implementaremos FF mediante Cauer 1 o remociones continuas en infinito
koo, YRC_cauer_oo, rem = cauer_RC(YRC, remover_en_inf=False)

print_subtitle('Implementación escalera de $Y_{RC}$ y $Z_{RL}$')

print_latex(a_equal_b_latex_s(a_equal_b_latex_s('$ Y_{RC}(s)', YRC)[1:-1], YRC_cauer_oo ))

# Tratamos a nuestra función inmitancia como una Z
dibujar_cauer_RC_RL(koo, z_exc = YRC_cauer_oo)

print_latex(a_equal_b_latex_s(a_equal_b_latex_s('$ Z_{RL}(s)', YRC)[1:-1], YRC_cauer_oo ))

# Tratamos a nuestra función inmitancia como una Y
dibujar_cauer_RC_RL(koo, y_exc = YRC_cauer_oo)


Ambas redes duales, tienen características en común como ser que ambas tienen capacitores en serie e inductores en derivación. Esto contrasta con las redes halladas mediante Cauer 1, donde observamos lo contrario: inductores en serie y capacitores en derivación. Está claro que Cauer 1 da lugar a redes que, en caso **que imaginemos un puerto de salida en el extremo derecho de la red**, serían filtros **pasabajo**, mientras que sintetizar una red mediante Cauer 2 da lugar a una red escalera **pasa-altos**.
